In [1]:
import pyupbit
import time
import datetime
import numpy as np
import pandas as pd
import torch
import warnings
warnings.filterwarnings(action='ignore')

from utils import add_variables

access_key = "xoAxfIrwkWwwFa9jaJuIJ1x4udHWWxxzRLCNdj3F"
secret_key = "VWnu7NfOucIThpsQdSDZPnRE3cZCJyne1Kzxx5yJ"

In [2]:
upbit = pyupbit.Upbit(access_key, secret_key)
krw_balance = int(upbit.get_balance("KRW"))
print(f"잔액 : {krw_balance}원") # 원화 잔고 조회

잔액 : 57945원


In [4]:
from model import Model
    
model = Model(input_size = 22, recurrent_hidden_size = 256, dense_hidden_size = 256, output_size = 1)
device = torch.device('cpu')
model.load_state_dict(torch.load('model.pt', map_location=device))
model.eval()

Model(
  (gru): GRU(22, 256, num_layers=3, batch_first=True, dropout=0.2, bidirectional=True)
  (dense1): Linear(in_features=512, out_features=256, bias=True)
  (bn1): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (dropout1): Dropout(p=0.3, inplace=False)
  (dense2): Linear(in_features=256, out_features=256, bias=True)
  (bn2): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (dropout2): Dropout(p=0.3, inplace=False)
  (out): Linear(in_features=256, out_features=1, bias=False)
)

In [6]:
from trader import TradingBot

ticker = "KRW-SBD"
bot = TradingBot(upbit, model, ticker = ticker, interval = "minute1", window_size = 20,
                 baseline = 0.02, lossbreak = False,
                 hour = 4, stoploss = -0.03)
bot.trade(krw = 50000)

하락장
5분 후 예상 수익률 : -1.329%
현재시간: 2021-05-26 07:47:51, 현재가: 9365.0, 평균단가: 9440.00, 수익률: -0.794%, 손익: -418원
기대 수익률 : 1.883%
보유상태: True, 동작상태: True



KeyboardInterrupt: 

In [ ]:
buy_sell_result = bot.get_trading_data()
print(buy_sell_result)
print()
print('총 수익: ', buy_sell_result['profit and loss'].sum(), '원')

In [6]:
import pandas as pd
import datetime
import time
import pyupbit
import matplotlib.pyplot as plt
import mplfinance as mpf

from utils import *
%matplotlib inline

class TradingChart:
    def __init__(self, dataframe, ticker, interval, sell_window = 3, buy_window = 5):
        
        """
        
        datrafrme : dataframe of buy and sell result
        ticker(str) : 
        interval : minute1, minute3, minute5, minute10, minute15, minute30, hour, day
        frm(str) :
    
        """
        
        self.dataframe = dataframe
        self.ticker = ticker
        self.interval = interval
        self.sell_window = sell_window
        self.buy_window = buy_window
    
    def plot(self):
        data = self.get_result_dataframe(self.dataframe)
       
        plt.figure(figsize = (20, 10))
        plt.plot(data.index, data['close'], color = 'k', label = 'Close Price')
        plt.plot(data.index, data['sell_ma'], color = 'c', label = 'Sell Line')
        plt.plot(data.index, data['buy_ma'], color = 'm', label = 'Buy Line')


        # plot 'buy' signals
        plt.plot(data[data['action'] == "매수"].index, data['close'][data['action'] == "매수"],
                 '^', markersize = 15, color = 'r', label = 'buy')


        # plot 'sell' signals
        plt.plot(data[data['action'] == "매도"].index, data['close'][data['action'] == "매도"],
                 'v', markersize = 15, color = 'b', label = 'sell')
        
        # plot 'lost' signals
        plt.plot(data[data['action'] == "손절"].index, data['close'][data['action'] == "손절"],
                 'o', markersize = 15, color = 'g', label = 'lost')
        
        
        
        plt.title('{} Trading Char'.format(self.ticker.split('-')[1]))
        plt.legend()
        plt.grid()
        plt.show()
        
    def ohlcv_chart(self):
        data = self.get_result_dataframe(self.dataframe)
        data = data[['open', 'high', 'low', 'close', 'volume']]
        
        kwargs = dict(title = '{} ohlcv Chart'.format(self.ticker.split('-')[1]), type = 'candle',
                      mav = (2, 4, 6), volume = True, ylabel = 'ohlc candles')
        mc = mpf.make_marketcolors(up = 'r', down = 'b', inherit = True)
        s = mpf.make_mpf_style(marketcolors = mc)
        mpf.plot(data, **kwargs, style = s)
        
    def candle_chart(self):
        data = self.get_result_dataframe(self.dataframe)
        data = data[['open', 'high', 'low', 'close', 'volume']]
        
        mpf.plot(data, title = '{} Candle Chart'.format(self.ticker.split('-')[1]), type = 'candle')
        
    def get_result_dataframe(self, dataframe):
        df1 = dataframe
        df2 = get_ohlcv_processed(df1, ticker = self.ticker, interval = self.interval,
                       sell_window = self.sell_window, buy_window = self.buy_window)
        
        data = df2.merge(df1, left_index = True, right_index = True, how = 'outer')
        
        return data